# scratch
## This notebook is for working out temporary issues and as a practice / experimentation repl
### Imports for reading in accounts

In [1]:
from finance_model.chart_of_accounts import ChartOfAccounts, unpickle_accounts
from finance_model.finance_enums import AccountLevel, CATEGORY, SUBCATEGORY, SUBCATEGORY2, IS_BS
import pandas as pd
from config import config

loading environment variables
cwd = C:\Coding\Finance-Model\notebooks
inside of notebook directory
   setting new working directory: C:\Coding\Finance-Model


### unpicle accounts

In [2]:
pickle = True

if pickle:
    accounts = unpickle_accounts()
else:
    accounts = ChartOfAccounts()
    accounts.account_map
    accounts.trial_balances
    

Total duration for function <unpickle_accounts> is 0.0022349999635480344


### Fill in the none values
#### (This code is not integrated into the main code base)
#### (remove after testing)

In [ ]:
# a = accounts.detailed_account_map[AccountLevel.SUBCATEGORY2].isna()
# accounts.detailed_account_map[a] = SUBCATEGORY2.NONE
# accounts.detailed_account_map.loc[a,AccountLevel.SUBCATEGORY2] = SUBCATEGORY2.NONE
# accounts.detailed_account_map
# a = accounts.trial_balances
# a = a.columns.to_list()
# a.sort()
# a

#### write_accounts to csv file to double check accuracy
#### (Integrate this into the command line)

In [ ]:
write_accounts = False
if write_accounts:
    accounts.write_accounts()

## Setup for manipulation of the trial balance dataframe
### imports and copy the trial balances

In [ ]:

tb = accounts.trial_balances.copy()
tb.T

---
### Group by year

In [ ]:
yearly = True
# yearly = False

if yearly:
    tb['year'] = [date[:4] for date in tb.index]
    grp = tb.groupby('year')
    # Choose the last of each grouping. This works because the data is cumulative
    tb = grp.tail(1).iloc[:, :-1]
tb.T

### Merge with detailed map

In [ ]:
tb = tb.T

tb[AccountLevel.ACCOUNT_NO] = tb.index
tb = tb.merge(accounts.detailed_account_map, how='left', on=AccountLevel.ACCOUNT_NO)
# tb

## Grouping by a single category

In [ ]:
grp = tb.groupby(AccountLevel.CATEGORY)
grp.first(2)
grp.get_group(CATEGORY.ASSETS)
a = grp.sum().iloc[:, :-6].T
# a[CATEGORY.ASSETS]

# a

In [ ]:
(a[CATEGORY.REVENUE] + a[CATEGORY.COGS] + a[CATEGORY.OPERATING_EXPENSES] + a[CATEGORY.OTHER_EXPENSES])/a[CATEGORY.REVENUE]
# grp.get_group(CATEGORY.LIABILITIES)

---
## Grouping by multiple categories
### Filter by balance sheet

In [ ]:

rows = tb[AccountLevel.IS_BS] == IS_BS.BS
tb2 = tb[rows]
# tb2
# grp2 = tb.loc[,:]

In [ ]:

grp2 = tb2.groupby([AccountLevel.CATEGORY, AccountLevel.SUBCATEGORY, AccountLevel.SUBCATEGORY2])
grp2
# grp.get_group(CATEGORY.EQUITY)

In [ ]:
# grp2.get_group((CATEGORY.ASSETS, SUBCATEGORY.CURRENT_ASSETS))
```for cat, df in grp2:
`    print(f'cat = {cat}')
    a = df
`    break
`space = ' ' * 3
# print(a)
print(f'Assets:')
print(f"{space * 1}Current Assets")
a = grp2.get_group((CATEGORY.ASSETS, SUBCATEGORY.CURRENT_ASSETS))
a.index=a[AccountLevel.ACCOUNT]
a = a.iloc[:, :-7]
# .iloc[:, :-7]
# print(a)
# a

## Tree


In [ ]:
from collections import OrderedDict, deque

class FinNode:
    def __init__(self, name):
        self.name = name
        self.children = OrderedDict()
    
    def insert_children(self, children: deque):
        if len(children) == 0:
            return
        child_name = children.popleft()
        if child_name == 'none':
            return
        if child_name in self.children.keys():
            child = self.children[child_name]
        else:
            child = FinNode(child_name)
            self.children[child_name] = child
        child.insert_children(children)
          
    def print(self, pad=''):
        print(f'{pad}{self.name}')
        for child in self.children.values():
            child.print(pad = pad + '     ')
        
top = FinNode('BS')
for cat, df in grp2:
    print(f'cat = {cat}')
    catlist = deque(cat)
    top.insert_children(catlist)
    
top.print()
    

## Filter transposed trial balances
### Filter by one category

In [ ]:
a = tb.loc[:,AccountLevel.CATEGORY:AccountLevel.ACCOUNT] == CATEGORY.ASSETS
b= a.sum(axis=1) == 1
tb[b]

In [ ]:
rows = accounts.account_map[AccountLevel.IS_BS] == IS_BS.BS
a = accounts.account_map.loc[rows, AccountLevel.CATEGORY: AccountLevel.SUBCATEGORY2]
a.iloc[0, :].to_list()

In [ ]:
tb

In [ ]:
len(tb)

In [ ]:
b == 1

In [ ]:
len(tb.columns)

---
# Groupby practice

In [ ]:
food_data = {
          "Item": ["Banana", "Cucumber", "Orange", "Tomato", "Watermelon"],
          "Type": ["Fruit", "Vegetable", "Fruit", "Vegetable", "Fruit"],
          "Price": [0.99, 1.25, 0.25, 0.33, 3.00]
        }
 
supermarket = pd.DataFrame(data = food_data)

supermarket

In [ ]:
groups = supermarket.groupby("Type")
groups

In [ ]:
groups["Price"].mean()

In [ ]:
groups.get_group("Vegetable")

In [ ]:
from enum import Enum

class cat_is(str, Enum):
    ONE = 'one'
    TWO = 'two'
    THREE = 'three'
    
class cat_bs(str, Enum):
    FOUR = 'four'
    FIVE = 'five'
    
class categoryj(cat_is, cat_bs):
    pass

In [ ]:
for i in cat_is:
    print(f'cat_is={i}')
for i in cat_bs:
    print(f'cat_bs={i}')
for i in category:
    print(f'category={i}')

In [ ]:
accounts

In [14]:
a = accounts.account_map.loc[:, AccountLevel.CATEGORY:AccountLevel.SUBCATEGORY2]
odict = {}
for i in range(len(a)):
    b = tuple(a.iloc[i, :])
    print(f'{i} = {b}')
    odict[b] = i
    

0 = (<CATEGORY.ASSETS: 'assets'>, <SUBCATEGORY.CURRENT_ASSETS: 'current assets'>, <SUBCATEGORY2.CASH_AND_EQUIVALENTS: 'cash and equivalents'>)
1 = (<CATEGORY.ASSETS: 'assets'>, <SUBCATEGORY.CURRENT_ASSETS: 'current assets'>, <SUBCATEGORY2.ACCOUNTS_RECEIVABLE: 'accounts receivable'>)
2 = (<CATEGORY.ASSETS: 'assets'>, <SUBCATEGORY.LONG_TERM_ASSETS: 'long term assets'>, <SUBCATEGORY2.PROPERTY_AND_EQUIPMENT: 'property and equipment'>)
3 = (<CATEGORY.ASSETS: 'assets'>, <SUBCATEGORY.LONG_TERM_ASSETS: 'long term assets'>, <SUBCATEGORY2.OTHER_ASSETS: 'other assets'>)
4 = (<CATEGORY.LIABILITIES: 'liabilities'>, <SUBCATEGORY.CURRENT_LIABILITIES: 'current liabilities'>, <SUBCATEGORY2.ACCOUNTS_PAYABLE: 'accounts payable'>)
5 = (<CATEGORY.LIABILITIES: 'liabilities'>, <SUBCATEGORY.CURRENT_LIABILITIES: 'current liabilities'>, <SUBCATEGORY2.ACCRUED_EXPENSES: 'accrued expenses'>)
6 = (<CATEGORY.LIABILITIES: 'liabilities'>, <SUBCATEGORY.CURRENT_LIABILITIES: 'current liabilities'>, <SUBCATEGORY2.OTHER_CU

In [4]:
AccountLevel.IS_BS

<AccountLevel.IS_BS: 'is_bs'>

In [6]:
accounts.account_map

,low,high,is_bs,normal balance,category,subcategory,subcategory2,sub_category3
0,11000,11999,BS,DEBIT,ASSETS,CURRENT_ASSETS,CASH_AND_EQUIVALENTS,NaN
1,12000,13000,BS,DEBIT,ASSETS,CURRENT_ASSETS,ACCOUNTS_RECEIVABLE,NaN
2,14700,16000,BS,DEBIT,ASSETS,LONG_TERM_ASSETS,PROPERTY_AND_EQUIPMENT,NaN
3,17500,19999,BS,DEBIT,ASSETS,LONG_TERM_ASSETS,OTHER_ASSETS,NaN
4,20000,20200,BS,CREDIT,LIABILITIES,CURRENT_LIABILITIES,ACCOUNTS_PAYABLE,NaN
5,20300,20399,BS,CREDIT,LIABILITIES,CURRENT_LIABILITIES,ACCRUED_EXPENSES,NaN
6,20400,20999,BS,CREDIT,LIABILITIES,CURRENT_LIABILITIES,OTHER_CURRENT_LIABILITIES,NaN
7,21000,21999,BS,CREDIT,LIABILITIES,LONG_TERM_LIABILITIES,ACCRUED_EXPENSES,NaN
8,22000,24999,BS,CREDIT,LIABILITIES,LONG_TERM_LIABILITIES,OTHER_LONG_TERM_LIABILITIES,NaN
9,27000,27099,BS,CREDIT,EQUITY,STOCK,NONE,NaN
